# TP Optimisation Différentiable 
<h3 align="right"> ENSAE, Avril 2018 </h3>


<h4 align="right"> Hicham Janati </h4>


In [1]:
import numpy as np
import scipy
from matplotlib import pyplot

## Méthode de Newton
### 1.1 Introduction
Soit $f$ une fonction de classe $\mathcal{C}^1$ de $\mathbb{R}^n$ dans $\mathbb{R}^n$. Le but de la méthode de Newton est de résoudre $f(x) = 0$.
Soit $x_0$ dans $\mathbb{R}^n$. L'approximation de premier ordre de $f$ dans un voisinage de $x_0$ est donnée par:
$$ \hat{f}(x) = f(x_0) + J(x_0)(x - x_0) $$
Oû $J$ est la matrice Jacobienne de f.
Annuler la tangeante $\hat{f}$ donne $x = x_0 - J^{-1}f(x_0)$ et obtient donc la suite d'itérés:
$$x_k = x_{k-1} - J^{-1}f(x_{k-1})$$


#### Question 1
Soit $x^{\star}$ un zéro de $f$. Supposons que $J(x^{\star})$ est inversible et que $f$ est de classe $\mathcal{C}^2$. Montrez que la méthode de Newton a une convergence localement quadratique i.e qu'il existe une boule B centrée en $x^{\star}$ telle que pour tout $x_0$ dans B, il existe $\alpha > 0$ telle que la suite de Newton vérifie: $$ \|x_{k+1} - x^{\star}\| < \alpha \|x_{k} - x^{\star}\|^2 $$
###### indication: Écrire l'approximation de deuxième ordre de f avec reste intégral.

---------

Deux remarques importantes:
- Newton est basée sur une approximation locale. La solution obtenue dépend donc du choix de $x_0$.
- $J$ doit être inversible.

---------

### 1.2 Optimisation sans contraintes
Soit $g$ une fonction de classe $\mathcal{C}^2$ de $\mathbb{R}^n$ dans $\mathbb{R}$.

##### Question 2 
Adapter la méthode de Newton pour résoudre $\min_{x \in \mathbb{R}^n} g(x)$.

 <br> <font color="green"> Comme il s'agit d'un problème sans contraintes, on peut appliquer la méthode de Newton à $\nabla g$ pour résoudre $\nabla g(x) = 0$. (A priori, on converge donc vers un point critique de $g$.) </font>
 
 <br> <font color="blue"> Pour les questions 3-4-5,  On prend $g: x \mapsto \frac{1}{2}\|Ax - b\|^2 + \gamma \|x\|^2$, avec $A \in \mathcal{M}_{m, n}(\mathbb{R})$, $b \in \mathbb{R}^m $ et $\gamma \geq 0 $</font>

##### Question 3
Donner le gradient et la hessienne de g et complétez les fonctions `gradient` et `hessian` ci-dessous.
Vérifiez votre gradient avec l'approximation numérique donnée par ```scipy.optimize.check_grad```.

<br> <font color="green"> Pour h dans un voisinage de x, on développe $g(x + h) = g(x) + \langle A^{\top}(Ax - b) + 2\gamma x, h\rangle + \frac{1}{2}h^{\top}(A^{\top}A + 2 \gamma I_n) h$.
 Ainsi, par identification de la partie linéaire en h: $\nabla g(x) = A^{\top}(Ax - b) + 2\gamma x$
 Et comme $A^{\top}A + 2 \gamma I_n$ est symmétrique: $\nabla^2 g(x) = A^{\top}A + 2 \gamma I_n$
 </font> 
   
##### Question 4
Lorsque $\gamma > 0$, montrez que la méthode de Newton converge en une itération indépendemment de $x_0$.
<br> <font color="green"> Il suffit d'écrire la condition d'optimalité $\nabla g(x) = 0$ qui donne exactement l'itération de Newton. 
 </font> 
 
##### Question 5

Complétez la fonction `newton` ci-dessous pour résoudre (2). Calculer l'inverse de la hessienne est très couteux (complexité $O(n^3)$), comment peut-on y remédier ?
Vérifiez le point (4) numériquement.

<br> <font color="green"> 
    Au lieu d'inverser la hessienne, on résout un système linéaire, ce qui est 3 fois moins couteux.

In [2]:
seed = 1729 # Seed du générateur aléatoire
m, n = 50, 100
rnd = np.random.RandomState(seed) # générateur aléatoire
A = rnd.randn(m, n) # une matrice avec des entrées aléatoires gaussiennes
b = rnd.randn(m) # on génére b aléatoirement également 
gamma = 1.

def g(x):
    """Compute the objective function g at a given x in R^n."""
    Ax = A.dot(x)
    gx = 0.5 * np.linalg.norm(Ax - b) ** 2 + gamma * np.linalg.norm(x) ** 2
    return gx

def gradient_g(x):
    """Compute the gradient of g at a given x in R^n."""
    # A faire
    g = A.T.dot(A.dot(x) - b) + 2 * gamma * x
    return g

def hessian_g(x):
    """Compute the hessian of g at a given x in R^n."""
    # A faire
    n = len(x)
    h = A.T.dot(A) + 2 * gamma * np.identity(n)
    return h


Vous pouvez vérifier que votre gradient est bon en utilisant la fonction de scipy `scipy.optimize.check_grad`. 
Exécutez ```scipy.optimize.check_grad?``` pour obtenir la documentation de la fonction.

In [3]:
from scipy.optimize import check_grad
x_test = rnd.randn(n) # point où on veut évaluer le gradient
check_grad(g, gradient_g, x_test) # compare gradient_g à des accroissements petis de g

0.00022141735630660996

In [4]:
def newton(x0, g=g, gradient=gradient_g, hessian=hessian_g, maxiter=10, verbose=True):
    """Solve min g with newton method"""
    
    x = x0.copy()
    if verbose:
        strings = ["Iteration", "g(x_k)", "max|gradient(x_k)|"]
        strings = [s.center(13) for s in strings]
        strings = " | ".join(strings)
        print(strings)

    for i in range(maxiter):
        H = hessian(x)
        d = gradient(x)
        
        if verbose:
            obj = g(x)
            strings = [i, obj, abs(d).max()] # On affiche des trucs 
            strings = [str(s).center(13) for s in strings]
            strings = " | ".join(strings)
            print(strings)
        
        # A faire
        x = x + np.linalg.solve(H, - d)

    return x

In [5]:
x0 = rnd.randn(n)
x = newton(x0)

  Iteration   |     g(x_k)    | max|gradient(x_k)|
      0       | 3570.345200187844 | 291.7910294967238
      1       | 1.0704068547572962 | 1.0200174038743626e-13
      2       | 1.0704068547572965 | 7.507883204027621e-15
      3       | 1.070406854757296 | 6.106226635438361e-15
      4       | 1.0704068547572962 | 4.884981308350689e-15
      5       | 1.070406854757296 | 4.810388198883686e-15
      6       | 1.0704068547572962 | 6.895525817007808e-15
      7       | 1.070406854757296 | 5.412337245047638e-15
      8       | 1.070406854757296 | 5.224987109642143e-15
      9       | 1.070406854757296 | 5.662137425588298e-15


### 1.3 Optimisation avec contraintes d'égalité

On s'intéresse à présent au problème avec contrainte linéaire: $$ \min_{\substack{x \in \mathbb{R}^n \\ Cx = d}} \frac{1}{2}\|Ax - b\|^2 + \gamma \|x \|^2 $$

##### Question 6
Donnez (en justifiant) le système KKT du problème.

<br> <font color="green"> 
    Notons la fonction objective par $g$ et les contraintes linéaires par $h(x) = Cx - d = 0$.
    Remearquez ici que h regroupe toutes les contraintes linéaires données par $\langle C_i, x\rangle - d_i = 0$ où l'indice i dénote la ième ligne. Notons chacune de ces contraintes par $h_i, i=1\dots p$

1) existence:
    Par continuité de h, l'ensemble des contraintes est un fermé. g est continue et clairement coercive, le minimum donc existe. 
    
2) convexité: 
    Comme $\gamma > 0$, on a pour tout $x,h  \in \mathbb{R}^n$: $$ h^{\top} \nabla^2 g(x)h = h^{\top}(A^{\top}A + 2\gamma I_n) = \|Ah\|^2 + 2 \gamma \|h\|^2 > 0$$ Donc g est strictivement convexe.
    La solution est donc unique. 
    
3) KKT:
    Les contraintes sont linéaires, elle est donc qualifié sur K, donc toute solution du problème vérifie KKT.
    Par convexité (+ qualification), KKT est aussi suffisante, donc toute solution de KKT est un minimum. 
    Par unicité, La solution de KKT est la solution du problème. Notons la $(x, \mu) \in \mathbb{R}^n \times \mathbb{R}^p$ :
    
   $$ \left\{ \begin{array}{l}
               \nabla g(x) + \sum_{i=1}^p \mu_i\nabla h_i(x) = 0 \\
               h(x) = 0
            \end{array}
            \right.  $$
   i.e 
      $$ \left\{ \begin{array}{l}
                A^{\top}(Ax - b) + 2\gamma x + C^{\top}\mu = 0 \\
                Cx - d = 0
            \end{array}
            \right.  $$
</font>

##### Question 7
Expliquer comment peut-on utiliser la méthode de Newton pour résoudre le système KKT.

<br> <font color="green"> 
Le problème est équivalent à son système KKT, on peut donc résoudre KKT avec la méthode de Newton appliquée à F ci-dessous pour résoudre $F(x, \mu) = 0$:
    $$ F(x, \mu) = \left(\nabla g(x) + \mu \nabla h(x), h(x)\right) $$ 
La suite de Newton s'écrit donc: $$(x_{k+1}, \mu_{k+1}) = (x_{k}, \mu_{k}) - J_F^{-1}(x_{k}, \mu_{k}) F(x_{k}, \mu_{k}) $$

On a donc besoin d'écrire la Jacobienne de F. 
On a:
$$ F(x, \mu) = \left(A^{\top}(Ax - b) + 2\gamma x + C^{\top}\mu, Cx - d\right) $$
On écrit la matricienne Jacobienne de F par blocs: 

$$ J_F(x, \mu) = \begin{pmatrix} A^{\top}A + 2\gamma I_n & C^{\top} \\ C & 0 \end{pmatrix} $$
 </font>
##### Question 8
Implémentez la fonction F dont on veut trouver un zéro et sa matrice Jacobienne.

##### Question 9
Implémentez la version de newton adaptée.

In [6]:
p = 5 # nombre de contraintes
C = rnd.randn(p, n)
d = rnd.randn(p)

def F(x, mu):
    """Compute the function F."""
    # On note f1 et f2 les composantes de F
    f1 = gradient_g(x) + C.T.dot(mu)
    f2 = C.dot(x) - d
    f = np.hstack((f1, f2))  # on concatene f1 et f2
    return f
    
def jac_F(x, mu):
    """Compute the jacobian of F."""
    # on crée une matrice de taille (n + p) x (n + p)
    J = np.zeros((n + p, n + p))
    J[:n, :n] = hessian_g(x)
    J[:n, n:] = C.T
    J[n:, :n] = C
    
    return J

def newton_constrained(xmu0, F=F, jac=jac_F, maxiter=10, verbose=True):
    """Solve constrained min g with newton method"""
    
    xmu = xmu0.copy()
    x = xmu[:n]
    mu = xmu[n:]
    if verbose:
        strings = ["Iteration", "max(abs(F(x_k, mu_k)))"]
        strings = [s.center(13) for s in strings]
        strings = " | ".join(strings)
        print(strings)

    for i in range(maxiter):
        J = jac(x, mu)
        f = F(x, mu)
        
        if verbose:
            strings = [i, abs(f).max()] # On affiche des trucs 
            strings = [str(s).center(13) for s in strings]
            strings = " | ".join(strings)
            print(strings)
        
        # A faire
        xmu = xmu + np.linalg.solve(J, - f)
        x = xmu[:n]
        mu = xmu[n:]
        
    return x

In [7]:
xmu0 = rnd.randn(n + p)
x = newton_constrained(xmu0)

  Iteration   | max(abs(F(x_k, mu_k)))
      0       | 159.44416332283464
      1       | 9.542019951958025e-14
      2       | 7.212980213111564e-15
      3       | 4.5449755070592346e-15
      4       | 7.369105325949477e-15
      5       | 7.271960811294775e-15
      6       | 6.04572815421367e-15
      7       | 6.25888230132432e-15
      8       | 4.246603069191224e-15
      9       | 6.300515664747763e-15


In [8]:
C.dot(x) - d

array([ 1.38777878e-16,  2.22044605e-16, -1.11022302e-16,  3.67761377e-16,
       -2.22044605e-16])